In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier,BaggingClassifier,RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score

In [3]:
df = pd.read_csv('/content/drive/My Drive/Data/creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
df.shape

(284807, 31)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [6]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.919560e-15,5.688174e-16,-8.769071e-15,2.782312e-15,-1.552563e-15,2.010663e-15,-1.694249e-15,-1.927028e-16,-3.137024e-15,1.768627e-15,9.170318e-16,-1.810658e-15,1.693438e-15,1.479045e-15,3.482336e-15,1.392007e-15,-7.528491e-16,4.328772e-16,9.049732e-16,5.085503e-16,1.537294e-16,7.959909e-16,5.367590e-16,4.458112e-15,1.453003e-15,1.699104e-15,-3.660161e-16,-1.206049e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,1.088850e+00,1.020713e+00,9.992014e-01,9.952742e-01,9.585956e-01,9.153160e-01,8.762529e-01,8.493371e-01,8.381762e-01,8.140405e-01,7.709250e-01,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,-2.458826e+01,-4.797473e+00,-1.868371e+01,-5.791881e+00,-1.921433e+01,-4.498945e+00,-1.412985e+01,-2.516280e+01,-9.498746e+00,-7.213527e+00,-5.449772e+01,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,-5.354257e-01,-7.624942e-01,-4.055715e-01,-6.485393e-01,-4.255740e-01,-5.828843e-01,-4.680368e-01,-4.837483e-01,-4.988498e-01,-4.562989e-01,-2.117214e-01,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,-9.291738e-02,-3.275735e-02,1.400326e-01,-1.356806e-02,5.060132e-02,4.807155e-02,6.641332e-02,-6.567575e-02,-3.636312e-03,3.734823e-03,-6.248109e-02,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,4.539234e-01,7.395934e-01,6.182380e-01,6.625050e-01,4.931498e-01,6.488208e-01,5.232963e-01,3.996750e-01,5.008067e-01,4.589494e-01,1.330408e-01,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,2.374514e+01,1.201891e+01,7.848392e+00,7.126883e+00,1.052677e+01,8.877742e+00,1.731511e+01,9.253526e+00,5.041069e+00,5.591971e+00,3.942090e+01,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [7]:
df.Class.value_counts(1)*100

0    99.827251
1     0.172749
Name: Class, dtype: float64

The target class is highly imbalanced which will need rebalancing.


In [10]:
y = df.Class
X = df.drop('Class', axis=1)

# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [11]:
# Modeling the data as is using Logistic Regression
# Train model
lr = LogisticRegression().fit(X_train, y_train)
 
# Predict on training set
lr_pred = lr.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [14]:
# Checking accuracy
print('Accuracy:',accuracy_score(y_test, lr_pred))

Accuracy: 0.9991011488441336


In [15]:
# Checking unique values
predictions = pd.DataFrame(lr_pred)
predictions[0].value_counts()

0    71096
1      106
Name: 0, dtype: int64

In [16]:
# f1 score
f1_score(y_test, lr_pred)

0.7168141592920354

In [17]:
# confusion matrix
pd.DataFrame(confusion_matrix(y_test, lr_pred))

,0,1
0,71057,25
1,39,81


In [18]:
# Recall
recall_score(y_test, lr_pred)

0.675

In [20]:
# train model
rfc = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)

# predict on test set
rfc_pred = rfc.predict(X_test)

accuracy_score(y_test, rfc_pred)

0.9995646189713772

In [21]:
# f1 score
print('F1 Score:',f1_score(y_test, rfc_pred),'\n')

# confusion matrix
print('Confusion Matrix:',pd.DataFrame(confusion_matrix(y_test, rfc_pred)),'\n')

# recall score
print('Recall:',recall_score(y_test, rfc_pred),'\n')

F1 Score: 0.8558139534883721 

Confusion Matrix:        0   1
0  71079   3
1     28  92 

Recall: 0.7666666666666667 



**A Significant improvement in the F1 score and Recall**

### Trying Resampling Techniques


In [22]:
# Oversampling - Creating copies of minority class

from sklearn.utils import resample

In [60]:
# Separate input features and target
y = df.Class
X = df.drop('Class', axis=1)

# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)

# concatenate our training data back together
X = pd.concat([X_train, y_train], axis=1)
X.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
264873,161634.0,-0.395578,1.489129,-0.833442,-0.224271,0.369444,-1.453886,0.796593,-0.060403,0.338270,-0.343885,-0.647990,-0.509490,-0.633729,-0.712156,0.765701,0.053830,0.526211,0.750505,-0.225921,0.054132,0.231624,0.955194,-0.172092,-0.041050,-0.313444,-0.174301,0.064657,-0.036960,2.74,0
163821,116237.0,1.950487,0.002312,-1.761814,1.232470,0.523175,-0.650657,0.504231,-0.200857,0.116805,0.443750,0.257633,0.358875,-1.313311,0.922108,-1.078538,-0.395038,-0.437151,0.074223,0.187086,-0.285207,0.086306,0.326297,-0.068839,-0.416589,0.426044,-0.486299,-0.031266,-0.072543,38.44,0
72083,54557.0,1.105167,-0.166253,0.569520,0.681043,-0.259189,0.642792,-0.437034,0.356746,0.441417,-0.113006,1.263824,1.017046,-0.791270,0.181033,-0.530067,-0.733662,0.436861,-0.903134,-0.242185,-0.226981,0.009073,0.293023,-0.028688,-0.242206,0.389813,0.482852,0.010705,-0.008399,1.00,0
196949,131771.0,1.805238,0.961264,-1.717212,4.094625,0.938666,-0.227785,0.152911,0.066753,-1.073784,0.334537,1.016183,-0.510443,-0.748733,-2.708662,-1.014069,2.220866,1.285249,1.619087,-1.602343,-0.150818,-0.137875,-0.450959,0.098530,-0.662272,-0.150154,-0.098852,-0.000030,0.017622,37.89,0
126213,77959.0,0.835421,-1.191847,0.578455,0.586101,-1.236663,0.194617,-0.532404,0.061561,-0.734344,0.754427,0.513247,0.704388,0.472193,-0.072043,-0.357247,-1.136932,-0.436168,2.047641,-0.775130,-0.045313,-0.072349,-0.109154,-0.308356,0.011968,0.461350,-0.244810,0.031845,0.060910,237.00,0


In [61]:
# separate minority and majority classes
not_fraud = X[X.Class==0]
fraud = X[X.Class==1]

# upsample minority
fraud_upsampled = resample(fraud,
                          replace=True, # sample with replacement
                          n_samples=len(not_fraud), # match number in majority class
                          random_state=1) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([not_fraud, fraud_upsampled])

# check new class counts
upsampled.Class.value_counts()

1    213245
0    213245
Name: Class, dtype: int64

In [62]:
# trying logistic regression again with the balanced dataset
y_train = upsampled.Class
X_train = upsampled.drop('Class', axis=1)

upsampled = LogisticRegression(solver='liblinear').fit(X_train, y_train)

upsampled_pred = upsampled.predict(X_test)

# Checking accuracy
accuracy_score(y_test, upsampled_pred)

0.981517373107497

In [51]:
# f1 score
print('F1 Score:',f1_score(y_test, upsampled_pred),'\n')

# confusion matrix
print('Confusion Matrix:',pd.DataFrame(confusion_matrix(y_test, upsampled_pred)),'\n')

# recall score
print('Recall:',recall_score(y_test, upsampled_pred),'\n')

F1 Score: 0.14766839378238342 

Confusion Matrix:        0     1
0  69772  1298
1     18   114 

Recall: 0.8636363636363636 



In [65]:
# train model
rfc = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)

# predict on test set
rfc_pred = rfc.predict(X_test)

# Checking accuracy
print('Accuracy Score:',accuracy_score(y_test, rfc_pred),'\n')

# f1 score
print('F1 Score:',f1_score(y_test, rfc_pred),'\n')

# recall score
print('Recall:',recall_score(y_test, rfc_pred),'\n')


# confusion matrix
print('Confusion Matrix:\n')
pd.DataFrame(confusion_matrix(y_test, rfc_pred))

Accuracy Score: 0.9995646189713772 

F1 Score: 0.8691983122362869 

Recall: 0.7803030303030303 

Confusion Matrix:



,0,1
0,71068,2
1,29,103


In [66]:
#Downsampling - Removing majority class rows

# still using our separated classes fraud and not_fraud from above

# downsample majority
not_fraud_downsampled = resample(not_fraud,
                                replace = False, # sample without replacement
                                n_samples = len(fraud), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled = pd.concat([not_fraud_downsampled, fraud])

# checking counts
downsampled.Class.value_counts()

1    360
0    360
Name: Class, dtype: int64

In [67]:
# trying logistic regression again with the undersampled dataset

y_train = downsampled.Class
X_train = downsampled.drop('Class', axis=1)

undersampled = LogisticRegression(solver='liblinear').fit(X_train, y_train)

undersampled_pred = undersampled.predict(X_test)

In [54]:
# Checking accuracy
print('Accuracy Score:',accuracy_score(y_test, undersampled_pred),'\n')

# f1 score
print('F1 Score:',f1_score(y_test, undersampled_pred),'\n')

# confusion matrix
print('Confusion Matrix:',pd.DataFrame(confusion_matrix(y_test, undersampled_pred)),'\n')

# recall score
print('Recall:',recall_score(y_test, undersampled_pred),'\n')

Accuracy Score: 0.9758574197354007 

F1 Score: 0.11710323574730355 

Confusion Matrix:        0     1
0  69369  1701
1     18   114 

Recall: 0.8636363636363636 



In [68]:
# train model
rfc = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)

# predict on test set
rfc_pred = rfc.predict(X_test)

# Checking accuracy
print('Accuracy Score:',accuracy_score(y_test, rfc_pred),'\n')

# f1 score
print('F1 Score:',f1_score(y_test, rfc_pred),'\n')

# recall score
print('Recall:',recall_score(y_test, rfc_pred),'\n')


# confusion matrix
print('Confusion Matrix:\n')
pd.DataFrame(confusion_matrix(y_test, rfc_pred))

Accuracy Score: 0.9793685570630039 

F1 Score: 0.13333333333333333 

Recall: 0.8560606060606061 

Confusion Matrix:



,0,1
0,69620,1450
1,19,113


In [69]:
#SMOTE or Synthetic Minority Oversampling Technique is a popular algorithm to creates sythetic observations of the minority class.

from imblearn.over_sampling import SMOTE

# Separate input features and target
y = df.Class
X = df.drop('Class', axis=1)

# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)

sm = SMOTE(random_state=27, ratio=1.0)
X_train, y_train = sm.fit_sample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [56]:
smote = LogisticRegression(solver='liblinear').fit(X_train, y_train)

smote_pred = smote.predict(X_test)

In [58]:
# Checking accuracy
print('Accuracy Score:',accuracy_score(y_test, smote_pred),'\n')

# f1 score
print('F1 Score:',f1_score(y_test, smote_pred),'\n')

# recall score
print('Recall:',recall_score(y_test, smote_pred),'\n')


# confusion matrix
print('Confusion Matrix:\n')
pd.DataFrame(confusion_matrix(y_test, smote_pred))

Accuracy Score: 0.9858571388444145 

F1 Score: 0.18461538461538463 

Recall: 0.8636363636363636 

Confusion Matrix:



,0,1
0,70081,989
1,18,114


In [70]:
# train model
rfc = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)

# predict on test set
rfc_pred = rfc.predict(X_test)

# Checking accuracy
print('Accuracy Score:',accuracy_score(y_test, rfc_pred),'\n')

# f1 score
print('F1 Score:',f1_score(y_test, rfc_pred),'\n')

# recall score
print('Recall:',recall_score(y_test, rfc_pred),'\n')


# confusion matrix
print('Confusion Matrix:\n')
pd.DataFrame(confusion_matrix(y_test, rfc_pred))

Accuracy Score: 0.9995365298727564 

F1 Score: 0.8685258964143426 

Recall: 0.8257575757575758 

Confusion Matrix:



,0,1
0,71060,10
1,23,109


In [80]:
NB=GaussianNB()
KNN=KNeighborsClassifier(n_neighbors=43,weights='distance')
DT=DecisionTreeClassifier(criterion='gini',max_depth=3,min_samples_leaf=8,random_state=0)
Bag=BaggingClassifier(n_estimators=26,random_state=0)
AB=AdaBoostClassifier(n_estimators=10,random_state=0)
ABL=AdaBoostClassifier(base_estimator=smote,n_estimators=50,random_state=0)
GB=GradientBoostingClassifier(n_estimators=59)

In [81]:
models = []
models.append(('NaiveBayes', NB))
models.append(('KNN',KNN))
models.append(('DecisionTree',DT))
models.append(('BaggingClassifier',Bag))
models.append(('AdaBoost',AB))
models.append(('BoostedLR',ABL))
models.append(('GBoost',GB))

In [82]:
for name, model in models:
    print('\n',name,':\n')
    m = model.fit(X_train, y_train)
    # predict on test set
    ypred = m.predict(X_test)
    
    # Checking accuracy
    print('Accuracy Score:',accuracy_score(y_test, ypred),'\n')
    
    # f1 score
    print('F1 Score:',f1_score(y_test, ypred),'\n')
    
    # recall score
    print('Recall:',recall_score(y_test, ypred),'\n')
    
    # confusion matrix
    print('Confusion Matrix:\n')
    print(pd.DataFrame(confusion_matrix(y_test, rfc_pred)))


 NaiveBayes :

Accuracy Score: 0.9927389680065166 

F1 Score: 0.27692307692307694 

Recall: 0.75 

Confusion Matrix:

       0    1
0  71060   10
1     23  109

 KNN :

Accuracy Score: 0.8956489986236341 

F1 Score: 0.018234672304439748 

Recall: 0.5227272727272727 

Confusion Matrix:

       0    1
0  71060   10
1     23  109

 DecisionTree :

Accuracy Score: 0.9892980534254656 

F1 Score: 0.22874493927125508 

Recall: 0.8560606060606061 

Confusion Matrix:

       0    1
0  71060   10
1     23  109

 BaggingClassifier :

Accuracy Score: 0.9988202578579254 

F1 Score: 0.7123287671232875 

Recall: 0.7878787878787878 

Confusion Matrix:

       0    1
0  71060   10
1     23  109

 AdaBoost :

Accuracy Score: 0.9772197410185107 

F1 Score: 0.12513484358144553 

Recall: 0.8787878787878788 

Confusion Matrix:

       0    1
0  71060   10
1     23  109

 BoostedLR :

Accuracy Score: 0.9850846886323418 

F1 Score: 0.1780185758513932 

Recall: 0.8712121212121212 

Confusion Matrix:

       0